In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [8]:
df = pd.read_csv("kc_house_data.csv")
df = df.drop(["id", "date"], axis=1)
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [16]:
X = df.drop("price", axis=1)
y = df["price"]

In [30]:
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = ["zipcode"]
numerical_cols.remove("zipcode")
numerical_cols

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [28]:
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['bedrooms', 'bathrooms', 'sqft_living',
                                  'sqft_lot', 'floors', 'waterfront', 'view',
                                  'condition', 'grade', 'sqft_above',
                                  'sqft_basement', 'yr_built', 'yr_renovated',
                                  'lat', 'long', 'sqft_living15',
                                  'sqft_lot15']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['zipcode'])])

In [36]:
X_processed = preprocessor.fit_transform(X)
X_processed

<21613x87 sparse matrix of type '<class 'numpy.float64'>'
	with 389034 stored elements in Compressed Sparse Row format>

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [42]:
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(1)  # Output layer for regression
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

In [48]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - loss: 429078118400.0000 - mae: 540066.8125 - val_loss: 388763910144.0000 - val_mae: 528579.4375
Epoch 2/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - loss: 386626125824.0000 - mae: 520968.5000 - val_loss: 326158254080.0000 - val_mae: 485296.0938
Epoch 3/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 336009199616.0000 - mae: 473476.1562 - val_loss: 220010184704.0000 - val_mae: 394916.5312
Epoch 4/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 207851159552.0000 - mae: 367649.6875 - val_loss: 128016949248.0000 - val_mae: 284808.2188
Epoch 5/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 126227095552.0000 - mae: 271521.9375 - val_loss: 86453886976.0000 - val_mae: 227820.0312
Epoch 6/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - loss: 86324142080.0000 - mae: 219764.3750 - val_loss: 70029729792.0000 - val_mae: 200542.9375
Epoch 7/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 70988406784.0000 - mae: 19494

In [50]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {test_loss:.2f}")
print(f"Test MAE: {test_mae:.2f}")

Test MSE: 38596571136.00
Test MAE: 117698.61
